In [1]:
#This is main program errorless 


import sounddevice as sd
import numpy as np
import speech_recognition as sr
from gtts import gTTS
import wave
import os
import playsound

# Initialize the recognizer
r = sr.Recognizer()

# Function to speak text
def SpeakText(command):
    try:
        tts = gTTS(text=command, lang='en')
        filename = "temp.mp3"
        tts.save(filename)
        playsound.playsound(filename)
        os.remove(filename)
    except Exception as e:
        print("An error occurred while using gTTS:", e)

# Audio parameters
fs = 44100  # Sample rate
duration = 5  # Duration to record in seconds

# Function to record audio using sounddevice
def record_audio(duration, fs):
    print("Recording...")
    recording = sd.rec(int(duration * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()  # Wait until the recording is finished
    print("Finished recording.")
    return recording

# Function to save recorded audio as a WAV file
def save_wav(filename, data, fs):
    print(f"Saving audio to {filename}...")
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)  # 2 bytes for int16
        wf.setframerate(fs)
        wf.writeframes(data.tobytes())
    print("Audio saved successfully.")

# File path for the WAV file
wav_filename = "recorded_audio.wav"

while True:
    try:
        # Record audio
        audio_data = record_audio(duration, fs)

        # Save the recorded audio to a WAV file
        save_wav(wav_filename, audio_data, fs)

        # Use the recognizer to process the audio
        print(f"Processing {wav_filename}...")
        with sr.AudioFile(wav_filename) as source:
            audio = r.record(source)

            # Attempt to recognize the speech with retries
            retries = 3
            for attempt in range(retries):
                try:
                    MyText = r.recognize_google(audio)
                    MyText = MyText.lower()
                    break  # Exit the loop if recognition is successful
                except sr.RequestError as e:
                    print(f"Could not request results; attempt {attempt+1}/{retries} failed: {e}")
                    if attempt < retries - 1:
                        print("Retrying...")
                except sr.UnknownValueError:
                    print("Could not understand audio")
                    break
            else:
                print("Failed to recognize speech after several attempts.")
                continue

        print("Did you say:", MyText)
        SpeakText(f"Did you say: {MyText}")
    
    except Exception as e:
        print("An error occurred:", e)

    # Optional: Add a delay before the next recording
    import time
    time.sleep(2)

    # Ask user if they want to continue
    user_input = input("Do you want to continue? (yes/no): ").strip().lower()
    if user_input != 'yes':
        break



Recording...
Finished recording.
Saving audio to recorded_audio.wav...
Audio saved successfully.
Processing recorded_audio.wav...
Did you say: hello


Do you want to continue? (yes/no):  No
